**Dataset**

In this homework, we will use the Car price dataset.

Download it with wget:
```bash
wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
```

We'll keep working with the MSRP variable, and we'll transform it to a classification task.

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

--2023-10-02 19:57:25--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1,4M) [text/plain]
Saving to: ‘data.csv.5’

data.csv.5          100%[===================>]   1,41M  3,10MB/s    in 0,5s    

2023-10-02 19:57:25 (3,10 MB/s) - ‘data.csv.5’ saved [1475504/1475504]



In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score, accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.feature_extraction import DictVectorizer

In [3]:
data = pd.read_csv('data.csv')
data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


**Features**

For the rest of the homework, you'll need to use only these columns:

   * Make,
   * Model,
   * Year,
   * Engine HP,
   * Engine Cylinders,
   * Transmission Type,
   * Vehicle Style,
   * highway MPG,
   * city mpg


In [4]:
features_to_use = ['Make','Model','Year','Engine HP','Engine Cylinders','Transmission Type',
                   'Vehicle Style','highway MPG','city mpg']+['MSRP']

In [5]:
data = data[features_to_use]

**Data preparation**
* Select only the features from above and transform their names using next line:
```bash
data.columns = data.columns.str.replace(' ', '_').str.lower()
```
* Fill in the missing values of the selected features with 0.
* Rename MSRP variable to price.

In [6]:
data.columns = data.columns.str.replace(' ', '_').str.lower()


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               11914 non-null  object 
 1   model              11914 non-null  object 
 2   year               11914 non-null  int64  
 3   engine_hp          11845 non-null  float64
 4   engine_cylinders   11884 non-null  float64
 5   transmission_type  11914 non-null  object 
 6   vehicle_style      11914 non-null  object 
 7   highway_mpg        11914 non-null  int64  
 8   city_mpg           11914 non-null  int64  
 9   msrp               11914 non-null  int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 930.9+ KB


In [8]:
data.fillna(0, inplace=True)

In [9]:
data.rename(columns={'msrp': 'price'}, inplace=True)

In [10]:
data.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


**Question 1**

What is the most frequent observation (mode) for the column transmission_type?

    * AUTOMATIC
    * MANUAL
    * AUTOMATED_MANUAL
    * DIRECT_DRIVE


In [11]:
print(f'The most frequent observation: {data["transmission_type"].mode().values[0]}')

The most frequent observation: AUTOMATIC


**Question 2**

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?
* engine_hp and year
* engine_hp and engine_cylinders
* highway_mpg and engine_cylinders
* highway_mpg and city_mpg <---this pair


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               11914 non-null  object 
 1   model              11914 non-null  object 
 2   year               11914 non-null  int64  
 3   engine_hp          11914 non-null  float64
 4   engine_cylinders   11914 non-null  float64
 5   transmission_type  11914 non-null  object 
 6   vehicle_style      11914 non-null  object 
 7   highway_mpg        11914 non-null  int64  
 8   city_mpg           11914 non-null  int64  
 9   price              11914 non-null  int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 930.9+ KB


In [13]:
numerical = ['engine_hp', 'engine_cylinders', 'year','highway_mpg','city_mpg']

In [14]:
data[numerical].corr()

,engine_hp,engine_cylinders,year,highway_mpg,city_mpg
engine_hp,1.000000,0.774851,0.338714,-0.415707,-0.424918
engine_cylinders,0.774851,1.000000,-0.040708,-0.614541,-0.587306
year,0.338714,-0.040708,1.000000,0.258240,0.198171
highway_mpg,-0.415707,-0.614541,0.258240,1.000000,0.886829
city_mpg,-0.424918,-0.587306,0.198171,0.886829,1.000000


**Make price binary**

* Now we need to turn the price variable from numeric into a binary format.
* Let's create a variable above_average which is 1 if the price is above its mean value and 0 otherwise.

**Split the data**

* Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

* Make sure that the target value (above_average) is not in your dataframe.


In [15]:
data['above_average'] = (data.price>data.price.mean()).astype(int)

In [16]:
data['above_average'].value_counts(normalize=True)

above_average
0    0.725617
1    0.274383
Name: proportion, dtype: float64

In [17]:
data_full_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

In [18]:
data_train, data_val = train_test_split(data_full_train, test_size=0.25, random_state=42)

In [19]:
data.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'price', 'above_average'],
      dtype='object')

**Question 3**

Calculate the mutual information score between **above_average** and other categorical variables in our dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).
Which of these variables has the lowest mutual information score?
* make
* model
* transmission_type <--this
* vehicle_style

In [20]:
categorical = ['make', 'model', 'transmission_type', 'vehicle_style']
for cat in categorical:
    print(f'For {cat} mutual information is {mutual_info_score(data_train.above_average, data_train[cat])}')

For make mutual information is 0.23976875439118345
For model mutual information is 0.46234389209653004
For transmission_type mutual information is 0.020957541896301862
For vehicle_style mutual information is 0.0841430195677944


**Question 4**

Now let's train a logistic regression.
Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
* To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
```python
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
```
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
What accuracy did you get?

    * 0.60
    * 0.72
    * 0.84
    * 0.95 <--- this

In [21]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

In [22]:
y_train = data_train.above_average

In [23]:
X_train = data_train.drop(columns=['above_average', 'price'])

In [24]:
y_val = data_val.above_average

In [25]:
X_val = data_val.drop(columns=['above_average', 'price'])

In [26]:
train_dict = X_train.to_dict(orient='records')
dv = DictVectorizer()
encoded_train = dv.fit_transform(train_dict)
encoded_val = dv.transform(X_val.to_dict(orient='records'))

In [27]:
model.fit(encoded_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [28]:
y_predicted = model.predict(encoded_val)

In [29]:
print(f'Accuracy {round(accuracy_score(y_val, y_predicted),2)}')

Accuracy 0.95


**Question 5**

Let's find the least useful feature using the feature elimination technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
* Which of following feature has the smallest difference?

    * year <--- this
    * engine_hp
    * transmission_type
    * city_mpg

In [30]:
columns_to_try = ['year', 'engine_hp', 'transmission_type', 'city_mpg']

In [39]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

_encoded_train_dict = dv.fit_transform(X_train[columns_to_try].to_dict(orient='records'))
_encoded_val_dict = dv.transform(X_val[columns_to_try].to_dict(orient='records'))

model.fit(_encoded_train_dict, y_train)
original_accuracy = accuracy_score(y_val, model.predict(_encoded_val_dict))

In [40]:
original_accuracy

0.8850188837599664

In [45]:
for col in columns_to_try:
    X_train_w_col = X_train[columns_to_try].drop(columns=[col])
    X_val_w_col = X_val[columns_to_try].drop(columns=[col])
   
    _encoded_train_dict = dv.fit_transform(X_train_w_col.to_dict(orient='records'))
    _encoded_val_dict = dv.transform(X_val_w_col.to_dict(orient='records'))
    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model.fit(_encoded_train_dict, y_train)
    
    y_predicted = model.predict(_encoded_train_dict)
    print(f'The difference between original accuracy and without {col}: {original_accuracy - accuracy_score(y_train, y_predicted)}')

The difference between original accuracy and without year: 0.014425710844465534
The difference between original accuracy and without engine_hp: 0.13725727212034688
The difference between original accuracy and without transmission_type: 0.015265106479608215
The difference between original accuracy and without city_mpg: 0.02421865992113037


**Question 6**

For this question, we'll see how to use a linear regression model from Scikit-Learn.
* We'll need to use the original column price. 
* Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data with a solver 'sag'. Set the seed to 42.
* This model also has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10].
* Round your RMSE scores to 3 decimal digits.
* Which of these alphas leads to the best RMSE on the validation set?
     * 0 <--- this
     * 0.01
     * 0.1
     * 1
     * 10

In [32]:
del data_train['above_average']
del data_val['above_average']

In [33]:
X_train_ridge = data_train.drop(columns=['price'])
y_train_ridge = data_train['price']

In [34]:
X_val_ridge = data_val.drop(columns=['price'])
y_val_ridge = data_val['price']

In [35]:
y_train_log = np.log(y_train_ridge)
y_val_log = np.log(y_val_ridge)

In [36]:
dv = DictVectorizer()
X_train_vec = dv.fit_transform(X_train_ridge.to_dict(orient='records'))
X_val_vec = dv.transform(X_val_ridge.to_dict(orient='records'))

In [37]:
for alpha in [0, 0.01, 0.1, 1, 10]:
    ridge = Ridge(alpha=alpha, solver='sag', random_state=42)
    ridge.fit(X_train_vec, y_train_log)
    y_val_predicted = ridge.predict(X_val_vec)
    print(f'For alpha {alpha}: RMSE = {round(mean_squared_error(y_val_log, y_val_predicted), 4)}')

For alpha 0: RMSE = 0.0631
For alpha 0.01: RMSE = 0.065
For alpha 0.1: RMSE = 0.0632
For alpha 1: RMSE = 0.0667
For alpha 10: RMSE = 0.1129
